In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [69]:
train_df = pd.read_csv('/Users/aidancurley/Documents/dsir/Submissions/Projects/project_2-master/datasets/train.csv')
test_df = pd.read_csv('/Users/aidancurley/Documents/dsir/Submissions/Projects/project_2-master/datasets/test.csv')

Data science is not a linear process. In this project, in particular, you will likely find that EDA, data cleaning, and exploratory visualizations will constantly feed back into each other. Here's an example:

- During basic EDA, you identify many missing values in a column/feature.  
- You consult the data dictionary and use domain knowledge to decide what is meant by this missing feature.  
- You impute a reasonable value for the missing value.  
- You plot the distribution of your feature.  
- You realize what you imputed has negatively impacted your data quality.  
- You cycle back, re-load your clean data, re-think your approach, and find a better solution.  

In [70]:
train_df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [114]:
id_df = train_df[['Id']]

In [115]:
nominal_column = ['MS SubClass', 'MS Zoning', 'Street', 'Alley', 'Land Contour', 'Lot Config', 'Neighborhood',
                 'Condition 1', 'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl', 'Exterior 1st',
                 'Exterior 2nd', 'Mas Vnr Type', 'Foundation', 'Heating', 'Central Air', 'Garage Type', 'Misc Feature',
                 'Sale Type']

In [189]:
# Merge Nominal categories
def cats(dataframe, column):
    for xi in column:
        if dataframe[xi].isna().sum() == 0:
            for n in column:
                colls = dataframe[['Id', n]]
                dums = pd.get_dummies(columns=[n], data=colls, drop_first=True)
                nominaldf = id_df.join(dums, lsuffix=='Id')
#         else:
#             statements = print(f"{column} needs work because NaN")
    
    return nominaldf
    

In [190]:
cats(train_df, nominal_column)

NameError: name 'lsuffix' is not defined

## MS SubClass (Nominal):

In [5]:
# MS SubClass
mssc_dummies = train_df[['Id','MS SubClass']]
ms_new = pd.get_dummies(columns=['MS SubClass'], data=mssc_dummies, drop_first=True)

In [6]:
ms_new

,Id,MS SubClass_30,MS SubClass_40,MS SubClass_45,MS SubClass_50,MS SubClass_60,MS SubClass_70,MS SubClass_75,MS SubClass_80,MS SubClass_85,MS SubClass_90,MS SubClass_120,MS SubClass_150,MS SubClass_160,MS SubClass_180,MS SubClass_190
0,109,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,544,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,153,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,318,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,255,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1587,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2047,785,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2048,916,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2049,639,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
X = ms_new[['MS SubClass_30', 'MS SubClass_40', 'MS SubClass_45', 'MS SubClass_50', 'MS SubClass_60', 'MS SubClass_70', 'MS SubClass_75', 'MS SubClass_80', 'MS SubClass_85', 'MS SubClass_90', 'MS SubClass_120', 'MS SubClass_150', 'MS SubClass_160', 'MS SubClass_180','MS SubClass_190']]
y = train_df['SalePrice']

In [8]:
def slr_model(X, y):
#      Step 1: Assemble our X and y variables
#      We need an X matrix that is n-by-p (in this case, p = 1)
#      We need a y vector that is length n
    
#     Instantiate the model
    model = LinearRegression()  
    
#     Fit the model
    model.fit(X, y) 
    
#     Find the coefficient
    score = model.score(X, y)
    
    y_pred = model.predict(X)

#     Take a peek at the model coefficient and intercept
    y_intercept = model.intercept_
    y_coef = model.coef_
    
    return y_pred, y_intercept, y_coef

In [9]:
y_pred, y_intercept, y_coef = slr_model(X, y)

In [10]:
metrics.mean_squared_error(y, y_pred, squared=False)

68317.74141069653

## MS Zoning (Nominal):

In [11]:
# MS Zoning
msz_dummies = train_df[['Id','MS Zoning']]
msz_new = pd.get_dummies(columns=['MS Zoning'], data=msz_dummies, drop_first=True)

In [12]:
msz_new.head(1)

,Id,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM
0,109,0,0,0,0,1,0


In [13]:
X = msz_new[['MS Zoning_C (all)', 'MS Zoning_FV', 'MS Zoning_I (all)', 'MS Zoning_RH', 'MS Zoning_RL', 'MS Zoning_RM']]
y = train_df['SalePrice']

In [14]:
def slr_model(X, y):
#      Step 1: Assemble our X and y variables
#      We need an X matrix that is n-by-p (in this case, p = 1)
#      We need a y vector that is length n
    
#     Instantiate the model
    model = LinearRegression()  
    
#     Fit the model
    model.fit(X, y) 
    
#     Find the coefficient
    score = model.score(X, y)
    
    y_pred = model.predict(X)

#     Take a peek at the model coefficient and intercept
    y_intercept = model.intercept_
    y_coef = model.coef_
    
    return y_pred, y_intercept, y_coef

In [15]:
y_pred, y_intercept, y_coef = slr_model(X, y)

In [16]:
metrics.mean_squared_error(y, y_pred, squared=False)

74718.9436958434

In [20]:
nominal = msz_new.merge(ms_new, on='Id')

## Street (Nominal):

In [25]:
street_dummies = train_df[['Id','Street']]
street_new = pd.get_dummies(columns=['Street'], data=street_dummies, drop_first=True)

In [26]:
street_new.head(1)

,Id,Street_Pave
0,109,1


In [27]:
X = street_new[['Street_Pave']]
y = train_df['SalePrice']

In [28]:
def slr_model(X, y):
#      Step 1: Assemble our X and y variables
#      We need an X matrix that is n-by-p (in this case, p = 1)
#      We need a y vector that is length n
    
#     Instantiate the model
    model = LinearRegression()  
    
#     Fit the model
    model.fit(X, y) 
    
#     Find the coefficient
    score = model.score(X, y)
    
    y_pred = model.predict(X)

#     Take a peek at the model coefficient and intercept
    y_intercept = model.intercept_
    y_coef = model.coef_
    
    return y_pred, y_intercept, y_coef
y_pred, y_intercept, y_coef = slr_model(X, y)
f"MSE is {metrics.mean_squared_error(y, y_pred, squared=False)}"

'MSE is 79045.84089947173'

In [30]:
nominal = nominal.merge(street_new, on='Id')

## Alley (Nominal):

In [ ]:
train_df['Alley'].isna().sum()

There are to many na values so disregard

## Land Contour (Nominal):

In [ ]:
train_df.head(1)

In [122]:
train_df['Land Contour'].unique()

array(['Lvl', 'HLS', 'Bnk', 'Low'], dtype=object)

In [123]:
train_df['Land Contour'].isna().sum()

0

In [124]:
land_dummies = train_df[['Id','Land Contour']]
land_new = pd.get_dummies(columns=['Land Contour'], data=land_dummies, drop_first=True)

In [125]:
land_new.head(1)

,Id,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl
0,109,0,0,1


In [ ]:
X = land_new[['Land Contour_HLS', 'Land Contour_Low', 'Land Contour_Lvl']]
y = train_df['SalePrice']

In [ ]:
def slr_model(X, y):
#      Step 1: Assemble our X and y variables
#      We need an X matrix that is n-by-p (in this case, p = 1)
#      We need a y vector that is length n
    
#     Instantiate the model
    model = LinearRegression()  
    
#     Fit the model
    model.fit(X, y) 
    
#     Find the coefficient
    score = model.score(X, y)
    
    y_pred = model.predict(X)

#     Take a peek at the model coefficient and intercept
    y_intercept = model.intercept_
    y_coef = model.coef_
    
    return y_pred, y_intercept, y_coef
y_pred, y_intercept, y_coef = slr_model(X, y)
f"MSE is {metrics.mean_squared_error(y, y_pred, squared=False)}"

In [126]:
nominal = nominal.merge(land_new, on='Id')

In [128]:
nominal.head(1)

,Id,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,MS SubClass_30,MS SubClass_40,MS SubClass_45,...,MS SubClass_90,MS SubClass_120,MS SubClass_150,MS SubClass_160,MS SubClass_180,MS SubClass_190,Street_Pave,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl
0,109,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


## Lot Config (Nominal):

In [ ]:
train_df.head(1)

In [129]:
train_df['Lot Config'].unique()

array(['CulDSac', 'Inside', 'Corner', 'FR2', 'FR3'], dtype=object)

In [130]:
train_df['Lot Config'].isna().sum()

0

In [131]:
lc_dummies = train_df[['Id','Lot Config']]
lc_new = pd.get_dummies(columns=['Lot Config'], data=lc_dummies, drop_first=True)

In [ ]:
lc_new.head(1)

In [ ]:
X = lc_new[['Lot Config_CulDSac', 'Lot Config_FR2', 'Lot Config_Inside']]
y = train_df['SalePrice']

In [ ]:
def slr_model(X, y):
#      Step 1: Assemble our X and y variables
#      We need an X matrix that is n-by-p (in this case, p = 1)
#      We need a y vector that is length n
    
#     Instantiate the model
    model = LinearRegression()  
    
#     Fit the model
    model.fit(X, y) 
    
#     Find the coefficient
    score = model.score(X, y)
    
    y_pred = model.predict(X)

#     Take a peek at the model coefficient and intercept
    y_intercept = model.intercept_
    y_coef = model.coef_
    
    return y_pred, y_intercept, y_coef
y_pred, y_intercept, y_coef = slr_model(X, y)
f"MSE is {metrics.mean_squared_error(y, y_pred, squared=False)}"

In [132]:
nominal = nominal.merge(lc_new, on='Id')

## Neighborhood (Nominal):

In [ ]:
train_df['Neighborhood'].unique()

In [133]:
train_df['Neighborhood'].isna().sum()

0

In [134]:
neigh_dummies = train_df[['Id','Neighborhood']]
neigh_new = pd.get_dummies(columns=['Neighborhood'], data=neigh_dummies, drop_first=True)

In [ ]:
neigh_list = neigh_new.columns

In [135]:
nominal = nominal.merge(neigh_new, on='Id')

## Condition 1 (Nominal):

In [136]:
train_df['Condition 1'].unique()

array(['RRAe', 'Norm', 'PosA', 'Artery', 'Feedr', 'PosN', 'RRAn', 'RRNe',
       'RRNn'], dtype=object)

In [137]:
train_df['Condition 1'].isna().sum()

0

In [138]:
c1_dummies = train_df[['Id','Condition 1']]
c1_new = pd.get_dummies(columns=['Condition 1'], data=c1_dummies, drop_first=True)

In [ ]:
c1_new.head()

In [ ]:
c1_list = c1_new.columns

In [ ]:
X = c1_new[c1_list]
y = train_df['SalePrice']

In [ ]:
def get_dummies(dataframe, dummy_col):
    dummy_df = dataframe[['Id', 'SalePrice', dummy_col]]
    dummy_df = pd.get_dummies(columns=[dummy_col], data=dummy_df, drop_first=True)
    model_X = dummy_df.drop(columns=['Id','SalePrice'])
    model_y = dummy_df['SalePrice']
    slr_model(model_X, model_y)

In [ ]:
get_dummies(train_df, 'Condition 1')

In [139]:
nominal = nominal.merge(c1_new, on='Id')

In [140]:
nominal.shape

(2051, 65)

## Condition 2 (Nominal):

In [141]:
train_df['Condition 2'].isna().sum()

0

In [148]:
def get_dummies(dataframe, dummy_col):
    dummy_df = dataframe[['Id', 'SalePrice', dummy_col]]
    dummy_df = pd.get_dummies(columns=[dummy_col], data=dummy_df, drop_first=True)
    dummy_df = dummy_df.drop(columns='SalePrice')
    return dummy_df

In [154]:
nominal = nominal.merge(get_dummies(train_df, 'Condition 2'), on='Id')

## Bldg Type (Nominal):

In [157]:
train_df['Bldg Type'].isna().sum()

0

In [158]:
train_df['Bldg Type'].unique()

array(['1Fam', 'TwnhsE', 'Twnhs', '2fmCon', 'Duplex'], dtype=object)

In [159]:
def get_dummies(dataframe, dummy_col):
    dummy_df = dataframe[['Id', 'SalePrice', dummy_col]]
    dummy_df = pd.get_dummies(columns=[dummy_col], data=dummy_df, drop_first=True)
    dummy_df = dummy_df.drop(columns='SalePrice')
    return dummy_df

In [160]:
nominal = nominal.merge(get_dummies(train_df, 'Bldg Type'))

## House Style (Nominal):

In [161]:
train_df['House Style'].isna().sum()

0

In [162]:
train_df['House Style'].unique()

array(['2Story', '1Story', '1.5Fin', 'SFoyer', 'SLvl', '2.5Unf', '2.5Fin',
       '1.5Unf'], dtype=object)

In [163]:
def get_dummies(dataframe, dummy_col):
    dummy_df = dataframe[['Id', 'SalePrice', dummy_col]]
    dummy_df = pd.get_dummies(columns=[dummy_col], data=dummy_df, drop_first=True)
    dummy_df = dummy_df.drop(columns='SalePrice')
    return dummy_df

In [164]:
nominal = nominal.merge(get_dummies(train_df, 'House Style'))

## Roof Style (Nominal):

In [165]:
train_df['Roof Style'].isna().sum()

0

In [166]:
train_df['Roof Style'].unique()

array(['Gable', 'Hip', 'Flat', 'Mansard', 'Shed', 'Gambrel'], dtype=object)

In [167]:
def get_dummies(dataframe, dummy_col):
    dummy_df = dataframe[['Id', 'SalePrice', dummy_col]]
    dummy_df = pd.get_dummies(columns=[dummy_col], data=dummy_df, drop_first=True)
    dummy_df = dummy_df.drop(columns='SalePrice')
    return dummy_df

In [169]:
nominal = nominal.merge(get_dummies(train_df, 'Roof Style'))

## Roof Matl (Nominal):

In [170]:
train_df['Roof Matl'].isna().sum()

0

In [172]:
train_df['Roof Matl'].unique()

array(['CompShg', 'WdShngl', 'Tar&Grv', 'WdShake', 'Membran', 'ClyTile'],
      dtype=object)

In [175]:
def get_dummies(dataframe, dummy_col):
    dummy_df = dataframe[['Id', 'SalePrice', dummy_col]]
    dummy_df = pd.get_dummies(columns=[dummy_col], data=dummy_df, drop_first=True)
    dummy_df = dummy_df.drop(columns='SalePrice')
    return dummy_df

In [176]:
nominal = nominal.merge(get_dummies(train_df, 'Roof Matl'))

## Exterior 1 (Nominal):

In [177]:
train_df['Exterior 1st'].isna().sum()

0

In [179]:
train_df['Exterior 1st'].unique()

array(['HdBoard', 'VinylSd', 'Wd Sdng', 'BrkFace', 'Plywood', 'MetalSd',
       'AsbShng', 'CemntBd', 'WdShing', 'Stucco', 'BrkComm', 'Stone',
       'CBlock', 'ImStucc', 'AsphShn'], dtype=object)

In [181]:
def get_dummies(dataframe, dummy_col):
    dummy_df = dataframe[['Id', 'SalePrice', dummy_col]]
    dummy_df = pd.get_dummies(columns=[dummy_col], data=dummy_df, drop_first=True)
    dummy_df = dummy_df.drop(columns='SalePrice')
    return dummy_df

In [182]:
nominal = nominal.merge(get_dummies(train_df, 'Exterior 1st'))

## Exterior 2 (Nominal):

In [191]:
train_df['Exterior 2nd'].isna().sum()

0

In [193]:
train_df['Exterior 2nd'].unique()

array(['Plywood', 'VinylSd', 'Wd Sdng', 'HdBoard', 'MetalSd', 'AsbShng',
       'CmentBd', 'Wd Shng', 'BrkFace', 'Stucco', 'Brk Cmn', 'ImStucc',
       'Stone', 'CBlock', 'AsphShn'], dtype=object)

In [194]:
def get_dummies(dataframe, dummy_col):
    dummy_df = dataframe[['Id', 'SalePrice', dummy_col]]
    dummy_df = pd.get_dummies(columns=[dummy_col], data=dummy_df, drop_first=True)
    dummy_df = dummy_df.drop(columns='SalePrice')
    return dummy_df

In [195]:
nominal = nominal.merge(get_dummies(train_df, 'Exterior 2nd'))

## Mas Vnr Type (Nominal):

## Foundation (Nominal):

In [198]:
train_df['Foundation'].isna().sum()

0

In [200]:
train_df['Foundation'].unique()

array(['CBlock', 'PConc', 'BrkTil', 'Slab', 'Stone', 'Wood'], dtype=object)

In [ ]:
def get_dummies(dataframe, dummy_col):
    dummy_df = dataframe[['Id', 'SalePrice', dummy_col]]
    dummy_df = pd.get_dummies(columns=[dummy_col], data=dummy_df, drop_first=True)
    dummy_df = dummy_df.drop(columns='SalePrice')
    return dummy_df

In [201]:
nominal = nominal.merge(get_dummies(train_df, 'Foundation'))

In [202]:
nominal.shape

(2051, 126)

## Heating (Nominal):

In [203]:
train_df['Heating'].isna().sum()

0

In [ ]:
def get_dummies(dataframe, dummy_col):
    dummy_df = dataframe[['Id', 'SalePrice', dummy_col]]
    dummy_df = pd.get_dummies(columns=[dummy_col], data=dummy_df, drop_first=True)
    dummy_df = dummy_df.drop(columns='SalePrice')
    return dummy_df

In [205]:
nominal = nominal.merge(get_dummies(train_df, 'Heating'))

## Central Air (Nominal):


In [206]:
train_df['Central Air'].isna().sum()

0

In [207]:
train_df['Central Air'].unique()

array(['Y', 'N'], dtype=object)

In [209]:
def get_dummies(dataframe, dummy_col):
    dummy_df = dataframe[['Id', 'SalePrice', dummy_col]]
    dummy_df = pd.get_dummies(columns=[dummy_col], data=dummy_df, drop_first=True)
    dummy_df = dummy_df.drop(columns='SalePrice')
    return dummy_df

In [210]:
nominal = nominal.merge(get_dummies(train_df, 'Central Air'))

## Garage Type (Nominal):

## Misc Feature (Nominal):

## Sale Type (Nominal):

In [213]:
train_df['Sale Type'].isna().sum()

0

In [214]:
train_df['Sale Type'].unique()

array(['WD ', 'New', 'COD', 'ConLD', 'Con', 'CWD', 'Oth', 'ConLI',
       'ConLw'], dtype=object)

In [215]:
def get_dummies(dataframe, dummy_col):
    dummy_df = dataframe[['Id', 'SalePrice', dummy_col]]
    dummy_df = pd.get_dummies(columns=[dummy_col], data=dummy_df, drop_first=True)
    dummy_df = dummy_df.drop(columns='SalePrice')
    return dummy_df

In [216]:
nominal = nominal.merge(get_dummies(train_df, 'Sale Type'))

In [217]:
nominal.shape

(2051, 139)

## Sale Condition (Nominal):

In [218]:
nominal.to_csv('nominal_df.csv', index=False)